In [112]:
import pandas as pd

brands = pd.read_csv('brands_cl.csv')
orderlines = pd.read_csv('orderlines_cl.csv')
orders = pd.read_csv('orders_cl.csv')
products = pd.read_csv('products_cl.csv')

In [114]:
[products.price.isnull().sum(), orderlines.unit_price.isnull().sum()]

[587, 0]

In [115]:
products.drop(columns='max_price_orderlines', inplace=True)
products = products.dropna(subset=['price'])
products.price.isnull().sum()

0

In [52]:
orderlines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price,check_orders
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,18.99,True
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,399.00,True
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,474.05,True
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,68.39,True
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,23.74,True


In [53]:
orders.head()

,order_id,created_date,total_paid,state,check_orders
0,241319,2017-01-02 13:35:40,44.99,Cancelled,True
1,241423,2017-11-06 13:10:02,136.15,Completed,True
2,242832,2017-12-31 17:40:03,15.76,Completed,True
3,243330,2017-02-16 10:59:38,84.98,Completed,True
4,243784,2017-11-24 13:35:19,157.86,Cancelled,True


In [116]:
ol_prices = (
orderlines
    .groupby(by=["sku"])
    .agg(min_ol_price=("unit_price", "min"),mean_ol_price=("unit_price", "mean"),max_ol_price=("unit_price", "max"))
    .reset_index()
)
ol_prices.head()
ol_prices.mean_ol_price.isnull().sum()

products01 = pd.merge(products, ol_prices, left_on="sku", right_on="sku", how='inner')
products01.head()
# products_01.mean_ol_price.isnull().sum()

,sku,name,desc,price,type,min_ol_price,mean_ol_price,max_ol_price
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,8696,0.00,46.276239,54.99
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,13855401,44.63,55.377605,60.49
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,1387,53.99,57.073226,59.00
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,1230,21.59,23.731739,25.00
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,1364,31.99,34.420000,35.71


In [117]:
# products01[products_01.price.isnull()]
# products01[products_01.mean_ol_price.isnull()]
# orderlines[orderlines.sku=="KIN0008"]

In [118]:
products01["price_diff"] = products01["price"] - products01["mean_ol_price"]
products01["discount_perc"] = 100-products01["mean_ol_price"]/products01["price"]*100
products01.sort_values(by="discount_perc", ascending=False).head(25)
[products01.shape, products01[products01.price_diff>0].shape, products01[products01.price_diff>0].shape[0]/products01.shape[0]*100]

[(7508, 10), (7172, 10), 95.5247735748535]

In [119]:
[products01.discount_perc.min(),products01.discount_perc.mean(),products01.discount_perc.max()]

[-626.8317200297842, 18.682177148144262, 90.03083590299191]

In [120]:
products01.describe()

,price,min_ol_price,mean_ol_price,max_ol_price,price_diff,discount_perc
count,7508.000000,7508.000000,7508.000000,7508.000000,7508.000000,7508.000000
mean,614.725685,487.234980,536.236356,577.410101,78.489329,18.682177
std,1200.337606,1034.002615,1079.162680,2149.384922,172.697807,18.159849
min,2.990000,-119.000000,2.115000,2.240000,-1302.000000,-626.831720
25%,39.990000,23.607500,30.784727,32.990000,6.077788,7.140118
50%,104.995000,68.990000,86.221176,90.990000,18.087866,15.754994
75%,579.990000,401.710000,473.166503,499.990000,67.882500,25.872289
max,15339.000000,13799.000000,14364.853333,159989.830000,2604.470000,90.030836
